<a href="https://colab.research.google.com/github/Aadhithan-aiengg/r1ragwithoutollama/blob/main/RagR1WithoutOllamaColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Step 1: Dependencies**

In [ ]:
!pip install -U langchain langchain-community
!pip install streamlit
!pip install PyPDF2
!pip install semantic-chunkers
!pip install open-text-embeddings
!pip install faiss
!pip install prompt-template
!pip install langchain
!pip install langchain_experimental
!pip install sentence-transformers
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 73.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.32
    Uninstalling langchain-core-0.3.32:
      Successfully uninstalled langchain-core-0.3.32
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.16
    Uninstalling langchain-0.3.16:
      Successfully uninstalled langchain-0.3.16
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

**Step 2: Main Program** *Remember to run this cell only once as it overloads the ram. If overloaded, restart the session and run this step in order for the last step to work.*

In [ ]:
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

print("welcome to deepseek r1 rag")
print("give it .txt or .pdf file to interact")
inputfile = input("enter file name")

loader = PDFPlumberLoader(inputfile)
docs = loader.load()

text_splitter = SemanticChunker(HuggingFaceEmbeddings())
documents = text_splitter.split_documents(docs)

embedder = HuggingFaceEmbeddings()
vector = FAISS.from_documents(documents, embedder)
retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 3})

model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,  # Use half-precision for faster inference
    device_map="auto",         # Automatically map model to GPU
    trust_remote_code=True
)

pipe = pipeline(
    "text-generation", model=model, tokenizer=tokenizer, max_new_tokens=2000, temperature=0.7, top_p=0.95,
    repetition_penalty=1.2
)
hf = HuggingFacePipeline(pipeline=pipe)

qa = RetrievalQA.from_chain_type(llm=hf, chain_type="refine", retriever=retriever, return_source_documents=False)

question = "give me the year of publishing of \'The Metamorphosis\'"
result = qa.run({"query": question})
print(result)

welcome to deepseek r1 rag
give it .txt or .pdf file to interact
enter file nameThe Metamorphosis.pdf


<ipython-input-2-f5d6cf297af2>:21: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  text_splitter = SemanticChunker(HuggingFaceEmbeddings())
<ipython-input-2-f5d6cf297af2>:21: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  text_splitter = SemanticChunker(HuggingFaceEmbeddings())
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a to

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Device set to use cuda:0
<ipython-input-2-f5d6cf297af2>:42: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  hf = HuggingFacePipeline(pipeline=pipe)
<ipython-input-2-f5d6cf297af2>:47: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa.run({"query": question})


The original question is as follows: give me the year of publishing of 'The Metamorphosis'
We have provided an existing answer: The original question is as follows: give me the year of publishing of 'The Metamorphosis'
We have provided an existing answer: Context information is below. 
------------
SYNOPSIS OF THE METAMORPHOSIS
The Metamorphosis is a classic of world literature, written by the
Czech Franz Kafka and published in 1915. It is an unusual story.
------------
Given the context information and not prior knowledge, answer the question: give me the year of publishing of 'The Metamorphosis'
<think>
Okay, so I need to figure out when The Meta Morrison was published. Let's start with what I know.

First, the user mentioned that it's a classic of world literature, authored by Czech author Franz Kafka from 1915. Wait, but isn't Kafka famous for "The Metamorphosis" as well? Maybe they're referring to two different works or something?

I should check if there are any other books named

**Optional but Required step if needed:** *modify only the 'question' part as per your requirement.*

In [6]:
question = "does Gregor Samsa transforms into an insect in \'The Metamorphism\'"
result = qa.run({"query": question})
print(result)

The original question is as follows: does Gregor Samsa transforms into an insect in 'The Metamorphism'
We have provided an existing answer: The original question is as follows: does Gregor Samsa transforms into an insect in 'The Metamorphism'
We have provided an existing answer: Context information is below. 
------------
C H A P T E R I
One morning, when Gregor Samsa woke from troubled dreams,
he found himself transformed in his bed into a horrible vermin. He lay on his armour-like back, and if he lifted his head a little
he could see his brown belly, slightly domed and divided by
arches into stiff sections. The bedding was hardly able to cover
it and seemed ready to slide off any moment. His many legs,
pitifully thin compared with the size of the rest of him, waved
about helplessly as he looked. "What's happened to me?" he thought. It wasn't a dream.
------------
Given the context information and not prior knowledge, answer the question: does Gregor Samsa transforms into an insect in